Opening the Data Sources

In [50]:
import pandas as pd
import matplotlib as plt
import numpy as np
import scipy as stats
import seaborn as sns
import zipfile
import sqlite3
import math

In [3]:
movie_gross_data = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
print(movie_gross_data)

                                            title      studio  domestic_gross  \
0                                     Toy Story 3          BV     415000000.0   
1                      Alice in Wonderland (2010)          BV     334200000.0   
2     Harry Potter and the Deathly Hallows Part 1          WB     296000000.0   
3                                       Inception          WB     292600000.0   
4                             Shrek Forever After        P/DW     238700000.0   
...                                           ...         ...             ...   
3382                                    The Quake       Magn.          6200.0   
3383                  Edward II (2018 re-release)          FM          4800.0   
3384                                     El Pacto        Sony          2500.0   
3385                                     The Swan  Synergetic          2400.0   
3386                            An Actor Prepares       Grav.          1700.0   

     foreign_gross  year  


In [4]:
#Finding missing values
movie_gross_data.isnull().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [5]:
#tmdb_movie_data = pd.read_csv("zippedData/tmdb.movies.csv.gz")
movie_budgets_data = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
#movie_info_data = pd.read_csv("zippedData/rt.movie_info.tsv.gz",sep="\t")

In [34]:
#movie_reviews_data = pd.read_csv("zippedData/rt.reviews.tsv.gz",sep="\t")

In [8]:
#tmdb_movie_data.isnull().sum()
movie_budgets_data.isnull().sum()
print(movie_budgets_data)
#missing a lot of values
#movie_info_data.isnull().sum()

      id  release_date                                        movie  \
0      1  Dec 18, 2009                                       Avatar   
1      2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2      3   Jun 7, 2019                                 Dark Phoenix   
3      4   May 1, 2015                      Avengers: Age of Ultron   
4      5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   
...   ..           ...                                          ...   
5777  78  Dec 31, 2018                                       Red 11   
5778  79   Apr 2, 1999                                    Following   
5779  80  Jul 13, 2005                Return to the Land of Wonders   
5780  81  Sep 29, 2015                         A Plague So Pleasant   
5781  82   Aug 5, 2005                            My Date With Drew   

     production_budget domestic_gross worldwide_gross  
0         $425,000,000   $760,507,625  $2,776,345,279  
1         $410,600,000   $241,063,8

In [81]:
wg = movie_budgets_data['worldwide_gross'].apply(lambda x: x.replace("$",""))
pb = movie_budgets_data['production_budget'].apply(lambda x: x.replace("$",""))
#percent_gain = wg *100/ pb-100
def p_gain(wg,pb):
    return wg*100/ pb-100
p_gain = p_gain.apply(p_gain)
print(p_gain)
#movie_budgets_data.add(percent_gain)

AttributeError: 'function' object has no attribute 'apply'

In [16]:
with zipfile.ZipFile("zippedData/im.db.zip", "r") as zObject:
        zObject.extractall()

In [17]:
conn = sqlite3.connect("im.db")
cur = conn.cursor()

In [18]:
pd.read_sql("""
SELECT *
FROM persons
;
""", conn).head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [ ]:
df = pd.read_sql("""SELECT * FROM movie_ratings, movie_basics ;""", conn)
df

In [8]:
df.isnull().sum()

name    0
dtype: int64